In [ ]:
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
!git clone https://github.com/alvaromoure/mmdetection.git
%cd mmdetection

!pip install -e .

In [ ]:
!pip install -q --upgrade wandb

# Imports

In [2]:
import wandb
import os
import shutil
import sys
import json
import torch,torchvision
from collections import defaultdict
from datetime import datetime
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config

import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector


import random
import numpy as np
from pathlib import Path

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [3]:
seed = 42
set_random_seed(seed, deterministic=False)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
wandb.login(key="")

# Constants

In [5]:
FOLD = 0
EPOCHS = 15

In [6]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = ''
EXP_NAME = f'faster_rcnn_Fold_{FOLD}'

In [12]:
train_ann_file = f'/working/coco_train_512x512_fold_{FOLD}.json'
val_ann_file = f'/working/coco_val_512x512_fold_{FOLD}.json'

# Model Training

In [7]:
model_name = f'faster_rcnn/faster_rcnn_x101_32x4d_fpn_1x_coco.py'

In [15]:
baseline_cfg_path = f'/kaggle/working/mmdetection/configs/{model_name}.py'
cfg = Config.fromfile(baseline_cfg_path)
job_folder = f'/kaggle/working/faster_rcnn'
if not os.path.exists(job_folder):
    os.makedirs(job_folder)
cfg.work_dir = job_folder


In [16]:
cfg.model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNeXt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://resnext101_32x4d'),
        groups=32,
        base_width=4),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=1,
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0, 0.0, 0.0, 0.0],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.5,
                neg_iou_thr=0.5,
                min_pos_iou=0.5,
                match_low_quality=False,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100)))

In [17]:
albu_train_transforms = [
    #dict(type='RandomResizedCrop',height=image_scale[0],width=image_scale[1],scale=(0.25,1.0),ratio=(0.75,1.3333),interpolation=1,p=1.0),
    dict(type='Rotate',limit=10,interpolation=1,border_mode=0,value=0,p=0.25),
    dict(type='HorizontalFlip',p=0.5),
    dict(
        type="OneOf",
        transforms=[
            dict(type="MotionBlur", p=0.2),
            dict(type="MedianBlur", p=0.1, blur_limit=3),
            dict(type='Blur',p=0.1,blur_limit=3)
        ],
        p=0.25),
    #dict(type='ShiftScaleRotate', shift_limit=0.0625,scale_limit=0.15, rotate_limit=15, p=0.4),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.2)
]

In [18]:
image_scale = (1333,800)
#image_scale = (512,512)

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=image_scale, keep_ratio=True),
    dict(type='EqualizeTransform', mode='cv', p=1,always_apply=True),
    dict(
        type='Albu',
        transforms=[dict(type='CLAHE', p=1),
                    *albu_train_transforms],
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]


In [19]:
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='Albu',
        transforms=[dict(type='CLAHE', p=1)],
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=False),
    dict(
        type='MultiScaleFlipAug',
        img_scale=image_scale,
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]


In [32]:
cfg.data = dict(
    samples_per_gpu=8,
    workers_per_gpu=2,
    train=dict(
        type='CocoDataset',
        ann_file=train_ann_file,
        img_prefix='/kaggle/input/siim-covid-19-256-512-768-1024-1280/images/train_512x512',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(type='EqualizeTransform',prob=1),
            dict(
                type='Albu',
                transforms=[dict(type='CLAHE', p=1),
                            *albu_train_transforms],
                keymap=dict(img='image', gt_bboxes='bboxes'),
                update_pad_shape=False,
                skip_img_without_anno=True,
                bbox_params=dict(
                    type='BboxParams',
                    format='pascal_voc',
                    label_fields=['gt_labels'],
                    filter_lost_elements=True,
                    min_visibility=0.1,
                    min_area=1)),
            dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=1),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
        ],
        classes=('Covid_Opacity', ),
        filter_empty_gt=True),
    val=dict(
        type='CocoDataset',
        ann_file=val_ann_file,
        img_prefix='/kaggle/input/siim-covid-19-256-512-768-1024-1280/images/train_512x512',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='EqualizeTransform',prob=1),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('Covid_Opacity', ),
        filter_empty_gt=True),
    test=dict(
        type='CocoDataset',
        ann_file=val_ann_file,
        img_prefix='data/images/train/',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='EqualizeTransform',prob=1),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('Covid_Opacity', ),
        filter_empty_gt=True))

In [33]:
cfg.evaluation = dict(interval=1, metric='bbox',save_best='bbox_mAP_50')
cfg.optimizer = dict(type='SGD', lr=1e-4, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8, 11])
cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)
cfg.checkpoint_config = dict(interval=1)

cfg.custom_hooks = [dict(type='NumClassCheckHook')]
cfg.dist_params = dict(backend='nccl')
cfg.log_level = 'INFO'
cfg.load_from = None
cfg.resume_from = None
cfg.workflow = [('train', 1)]
cfg.opencv_num_threads = 0
cfg.mp_start_method = 'fork'
cfg.auto_scale_lr = dict(enable=False, base_batch_size=16)
cfg.work_dir = '/kaggle/working/faster_rcnn/faster_rcnn_x101_32x4d_fpn_1x_coco'
cfg.seed = 42
cfg.gpu_ids = [0]
cfg.total_epochs = EPOCHS
cfg.classes = ('Covid_Opacity', )
cfg.evaluation_metric = 'bbox'
cfg.evaluation_interval = 1

### Logger Configuration

In [34]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project=PROJECT_NAME,
                                              name=f'{EXP_NAME}_{dt_string}',
                                              job_type='model_train',
                                              entity='alvaromoureupm',
                                             notes=f'Notebook: {NOTEBOOK_NAME}',
                                             config=cfg._cfg_dict.to_dict()),
                             log_artifact=True,
                             interval = 100
                            )
                       ]

In [36]:
cfg.dump('/kaggle/working/faster_rcnn/faster_rcnn.py')

In [ ]:
!python tools/train.py /kaggle/working/faster_rcnn/faster_rcnn.py